# QLand || QArdh

### Convert The Job Schedualing To QAOA
This project is undertaken for the NYU Quantum Computing Hackathon represents an innovative intersection of quantum computing technology and environmental science, specifically targeting the problem of desertification. Desertification is the degradation of land in arid, semi-arid, and dry sub-humid areas, primarily caused by various factors including climate change and human activities such as deforestation, unsustainable agricultural practices, and water misuse. To better monitor and predict patterns of desertification, the project utilizes a quantum computing approach, complemented by classical machine learning techniques.

### Overarching goal of the QLand | QArdh project

Enhance the capability to monitor and predict desertification more accurately and efficiently using both classical and quantum computational approaches. By improving the predictive models and optimizing computational resources, the project aims to contribute to the broader efforts in combating land degradation and promoting sustainable land management practices.

This detailed focus on combining advanced computational techniques promises a new frontier in environmental science, particularly in addressing critical global challenges such as desertification.


### Data Collection and Preprocessing
1. **Data Sets**: The data sets for this project are derived from an analysis focused on desertification in Iraq, incorporating resources like the work conducted by Omdena on using machine learning and satellite data for desertification detection. The data sets include measurements and indices from satellite imagery, primarily focusing on the Normalized Difference Vegetation Index (NVDI) and Land Surface Temperature (LST), along with other relevant environmental indicators.

### Classical Machine Learning Models
2. **Classical Analysis**: Initially, the project employs classical machine learning models such as Random Forests (RF) and Support Vector Machines (SVM). These models are applied to subsets of environmental factors, specifically NVDI and LST, to establish baseline desertification predictions and to understand the influence of each individual factor on the desertification process.


### Quantum Job Scheduling Problem
In quantum computing, the job scheduling problem involves determining the optimal sequence of operations or tasks on quantum computers, which is crucial given the nascent and resource-constrained nature of current quantum technology. Quantum Approximate Optimization Algorithm (QAOA) is used here to address this scheduling problem.
  

### Set-Up Imports Needed 

In [1]:
from qiskit import Aer, execute
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit import algorithms
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from qiskit.utils import QuantumInstance

### Instantiating the quadratic program

In [7]:
jc = 3  # job count
mc = 2  # machine count
tc = 2  # timeslot count

# Define the Quadratic Program
qp = QuadraticProgram()

In [8]:
# Add binary variables for each job-timeslot combination
for i in range(1, mc+1):  # for mc number of machines
    for j in range(1, jc+1):  # for jc number of jobs
        for k in range(1,tc+1): # for tc number of timeslots
            qp.binary_var(f'machine{i}job{j}time{k}')

In [9]:
# Define the objective function (minimize the total duration) and initialize all variables to be 1
linear = {}
for i in range(1, mc+1):
    for j in range(1, jc+1):
        for k in range(1, tc+1):
            linear [f'machine{i}job{j}time{k}'] = 1
qp.minimize(linear=linear)

### Constraints

Putting constraints is a key aspect of ensuring the integrity of the execution and optimizing the scheduling of jobs. By enforcing that each job must be assigned to exactly one timeslot and only one job can be assigned to each timeslot, collisions are prevented, maintaining execution integrity. These constraints also aid in minimizing the total execution time by encouraging even distribution of workload across timeslots and efficient resource utilization. As a result, the scheduling algorithm can find solutions that optimize execution duration, enhancing efficiency and productivity.

In [10]:
# execute as much jobs as possible in the nearest time possible
for i in range(1, tc+1):
    job_vars = {f'machine{j}job{k}time{i}': 1 for j in range(1, mc+1) for k in range(1, jc+1)}
    rhs= max(min(jc-(mc*(i-1)), mc), 0)
    qp.linear_constraint(linear=job_vars, sense='==', rhs= rhs)

# ensure that all jobs are ran exactly once
for i in range(1, jc+1):
    job_vars = {f'machine{k}job{i}time{j}': 1 for k in range(1, mc+1) for j in range(1, tc+1)}
    qp.linear_constraint(linear=job_vars, sense='==', rhs=1)

# no intersection of job execution on the same machine and time
for i in range(1, mc+1):
    for j in range(1, tc+1):
        job_vars = {f'machine{i}job{k}time{j}': 1 for k in range(1, jc+1)}
        qp.linear_constraint(linear=job_vars, sense='<=', rhs=1)

 **QAOA**: This is an algorithm used in quantum computing to solve combinatorial problems and optimization challenges. It works by encoding a problem into a Hamiltonian, which describes the energy landscape of a quantum system. QAOA iteratively applies quantum gates to prepare states that approximate the solution to the optimization problem.
   - **Application in QArdh**: By using QAOA, the QArdh project aims to optimize the computational workflows required for analyzing the various data streams and models efficiently. This is particularly important when dealing with large-scale data like satellite images and complex interdependencies between environmental factors affecting desertification.

The following cell computes the solution while complying with the defined constraints, the output format is a key-value pair, where the key is the combination of the machine, job and timeslot and the value represents a boolean value that indicates whether or not to executing the job at that time and machine.

In [11]:
# Set up the QAOA algorithm
qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))

# Use the MinimumEigenOptimizer to solve the QUBO with QAOA
optimizer = MinimumEigenOptimizer(qaoa)
result = optimizer.solve(qp)

print(result)

/tmp/ipykernel_36414/450798756.py:2: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))
/tmp/ipykernel_36414/450798756.py:2: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(quantum_instance=QuantumInstance(Aer.get_backend('qasm_simulator')))


fval=3.0, machine1job1time1=0.0, machine1job1time2=0.0, machine1job2time1=1.0, machine1job2time2=0.0, machine1job3time1=0.0, machine1job3time2=1.0, machine2job1time1=1.0, machine2job1time2=0.0, machine2job2time1=0.0, machine2job2time2=0.0, machine2job3time1=0.0, machine2job3time2=0.0, status=SUCCESS
